In [1]:
!gdown  1mQ4JlkmLzPHo6-LK3QC71iK6JkTyke5i
!unzip -q "Copy of public_train.zip"

Downloading...
From: https://drive.google.com/uc?id=1mQ4JlkmLzPHo6-LK3QC71iK6JkTyke5i
To: /content/Copy of public_train.zip
100% 23.5G/23.5G [04:36<00:00, 84.9MB/s]


In [2]:
!pip install -qq paddlehub paddlepaddle-gpu

     |████████████████████████████████| 212 kB 24.7 MB/s 
     |████████████████████████████████| 394.0 MB 25 kB/s 
     |████████████████████████████████| 4.9 MB 60.4 MB/s 
     |████████████████████████████████| 1.6 MB 57.7 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 2.9 MB 58.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 365 kB 57.1 MB/s 
     |████████████████████████████████| 112 kB 48.0 MB/s 
     |████████████████████████████████| 1.3 MB 63.9 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 120 kB 75.9 MB/s 
     |████████████████████████████████| 212 kB 71.7 MB/s 
     |████████████████████████████████| 127 kB 71.7 MB/s 
     |████████████████████████████████| 204 kB 75.2 MB/s 
     |████████████████████████████████| 394 kB 68.0 MB/s 
     |█████████████

In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import cv2
import paddlehub as hub
import torch
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import random
from tqdm import tqdm
import pandas as pd


In [2]:
model = hub.Module(name='U2Net')

def infer(image):
  result = model.Segmentation(
      images=[image],
      batch_size=1,
      )
  return result[0]['front'][:,:,::-1], result[0]['mask']


Download https://bj.bcebos.com/paddlehub/paddlehub_dev/U2Net.tar.gz
[##################################################] 100.00%
Decompress /root/.paddlehub/tmp/tmptwo_fwst/U2Net.tar.gz
[##################################################] 100.00%


[2022-08-28 14:52:08,367] [    INFO] - Successfully installed U2Net-1.0.0


In [3]:
def checkBox(x,y,w,h,orgW,orgH):
  ratio = 0.25
  x1=x
  x2=x+w
  y1=y
  y2=y+h
  left,right,up,down=True,True,True,True
  if x1/orgW>ratio:
    left=False
  if (orgW-x2)/orgW>ratio:
    right=False
  if y1/orgH>ratio:
    up=False
  if (orgH-y2)/orgH>ratio:
    down=False
  return left,right,up,down

In [4]:
def check(image):
  image = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(20,20))
  plt.subplot(141)
  plt.imshow(image)
  plt.title('Original')
  plt.axis('off')
  _,img2=infer(image)
  ret,img2 = cv2.threshold(img2,127,255,cv2.THRESH_BINARY)
  plt.subplot(142)
  plt.imshow(img2,cmap='gray')
  plt.axis('off')
  plt.title('U2Net mask')
  plt.subplot(143)
  (cnts, _) = cv2.findContours(img2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cc=cv2.drawContours(image.copy(), cnts, -1, (0, 255, 0), 2)
  plt.imshow(cc,cmap='gray')
  plt.axis('off')
  plt.title('Contours')
  plt.subplot(144)
  if len(cnts):
    c = max(cnts, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    print(checkBox(x,y,w,h,image.shape[1],image.shape[0]),all(checkBox(x,y,w,h,image.shape[1],image.shape[0])))
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
  plt.imshow(image,cmap='gray')
  plt.axis('off')
  plt.title('New bouding box')
  plt.show()

In [5]:
# find bbox theo kiểu lấy tất cả các vùng rồi chọn min max
def findBBOX(image):
  _,img2=infer(image)
  _,img2 = cv2.threshold(img2,127,255,cv2.THRESH_BINARY)
  (cnts, _) = cv2.findContours(img2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  xMin,yMin,xMax,yMax = 1e8,1e8,0,0
  for cnt in cnts:
      x,y,w,h = cv2.boundingRect(cnt)
      xMin = min(x,xMin)
      yMin = min(y,yMin)
      xMax = max(x+w,xMax)
      yMax = max(y+h,yMax)
  if xMin<xMax and yMin<yMax:
    return xMin,yMin,xMax-xMin,yMax-yMin
  else:
    return -1,-1,-1,-1

In [6]:
# find theo kiểu lấy vùng có diện tích lớn nhất
def findBBOX(image):
  _,img2=infer(image)
  _,img2 = cv2.threshold(img2,127,255,cv2.THRESH_BINARY)
  (cnts, _) = cv2.findContours(img2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  x,y,w,h = -1,-1,-1,-1
  if len(cnts) != 0:
    c = max(cnts, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
  return x,y,w,h

In [7]:
image_paths = glob('/content/public_train/pill/image/*')
len(image_paths)

9502

Test

In [8]:
image_path = random.choice(image_paths)
print(image_path)
img = cv2.imread(image_path)
org = img.copy()
label_path = image_path.replace('image','label').replace('jpg','json')
label = pd.read_json(label_path)
for i,row in label.iterrows():
  x = row['x']
  y = row['y']
  w = row['w']
  h = row['h']
  crop_img = img[y:y+h, x:x+w]
  check(crop_img)
  nx,ny,nw,nh=findBBOX(crop_img)
  cv2.rectangle(org, (x,y), (x+w,y+h), (0, 0, 255), 1)
  if nw !=-1 and nh != -1 and all(checkBox(nx,ny,nw,nh,crop_img.shape[1],crop_img.shape[0])):
    nx+=x
    ny+=y
    cv2.rectangle(org, (nx,ny), (nx+nw,ny+nh), (0, 255, 0), 1)
plt.figure(figsize=(20,20))
plt.imshow(cv2.cvtColor(org,cv2.COLOR_BGR2RGB))
plt.show()

Output hidden; open in https://colab.research.google.com to view.

Convert all

In [9]:
!mkdir new_label

In [10]:
for image_path in tqdm(image_paths):
  img = cv2.imread(image_path)
  label_path = image_path.replace('image','label').replace('jpg','json')
  label_name = label_path.split('/')[-1]
  label = pd.read_json(label_path)
  for i,row in label.iterrows():
    x = row['x']
    y = row['y']
    w = row['w']
    h = row['h']
    crop_img = img[y:y+h, x:x+w]
    nx,ny,nw,nh=findBBOX(crop_img)
    if nw !=-1 and nh != -1 and all(checkBox(nx,ny,nw,nh,crop_img.shape[1],crop_img.shape[0])):
        nx+=x
        ny+=y
        label.at[i,'x']=nx
        label.at[i,'y']=ny
        label.at[i,'w']=nw
        label.at[i,'h']=nh
  label.to_json(f'new_label/{label_name}',orient="records")


100%|██████████| 9502/9502 [1:03:22<00:00,  2.50it/s]


In [11]:
!zip -r new_label.zip new_label

Streaming output truncated to the last 5000 lines.
  adding: new_label/VAIPE_P_1160_23.json (deflated 50%)
  adding: new_label/VAIPE_P_381_8.json (stored 0%)
  adding: new_label/VAIPE_P_402_1.json (deflated 2%)
  adding: new_label/VAIPE_P_1159_32.json (deflated 45%)
  adding: new_label/VAIPE_P_235_9.json (deflated 53%)
  adding: new_label/VAIPE_P_655_5.json (deflated 25%)
  adding: new_label/VAIPE_P_180_18.json (deflated 28%)
  adding: new_label/VAIPE_P_703_8.json (stored 0%)
  adding: new_label/VAIPE_P_114_7.json (deflated 38%)
  adding: new_label/VAIPE_P_676_1.json (deflated 42%)
  adding: new_label/VAIPE_P_168_27.json (stored 0%)
  adding: new_label/VAIPE_P_829_0.json (deflated 37%)
  adding: new_label/VAIPE_P_1164_13.json (deflated 38%)
  adding: new_label/VAIPE_P_230_7.json (deflated 39%)
  adding: new_label/VAIPE_P_212_8.json (deflated 56%)
  adding: new_label/VAIPE_P_578_2.json (deflated 46%)
  adding: new_label/VAIPE_P_221_30.json (deflated 48%)
  adding: new_label/VAIPE_P_414_

In [12]:
!cp new_label.zip /content/drive/MyDrive/new_label.zip